In [1]:
# !pip install pandas
# !pip install vomeet-client

In [2]:
import os
import sys
import pandas as pd
from IPython.display import clear_output, display
import time

# Add the testing directory to the path
sys.path.append('../testing')

from core import create_user_client, request_bot,get_transcript

from vomeet_client.vomeet import parse_url


In [3]:
# creates test user and returns client
client = create_user_client(base_url="http://localhost:18056",admin_api_key='token')

In [4]:
### API endpoints validate against the schemas

In [5]:
#Google Meet:
meeting_url="https://meet.google.com/fjp-cbnn-hug"
platform, native_meeting_id, passcode = parse_url(meeting_url)

In [7]:
#MS Teams:

meeting_url="https://teams.live.com/meet/9399697580372?p=A5AU2r3RVJDzsCajcA"
platform, native_meeting_id, passcode = parse_url(meeting_url)


### request bot

In [6]:
# invalid native_meeting_id
try:
    meeting_info = client.request_bot(
        platform=platform,
        native_meeting_id=meeting_url, # invalid native_meeting_id
        bot_name="Vomeet's bot",
        language='es',
        task = 'transcribe'
    )
except Exception as e:
    print(e)
#'msg': msg': "Google Meet ID must be in format 'abc-defg-hij' 

HTTP Error 422 for POST http://localhost:18056/bots: [{'loc': ['body', 'native_meeting_id'], 'msg': "Google Meet ID must be in format 'abc-defg-hij' (lowercase letters only)", 'type': 'value_error'}]


In [7]:
# invalid platform
try:
    meeting_info = client.request_bot(
        platform="invalid platform",
        native_meeting_id=native_meeting_id,
        bot_name="Vomeet's bot",
        language='es',
        task = 'transcribe'
    )
except Exception as e:
    print(e)
#'msg': "value is not a valid enumeration member; permitted: 'google_meet', 'zoom', 'teams'"

HTTP Error 422 for POST http://localhost:18056/bots: [{'loc': ['body', 'platform'], 'msg': "value is not a valid enumeration member; permitted: 'google_meet', 'zoom', 'teams'", 'type': 'type_error.enum', 'ctx': {'enum_values': ['google_meet', 'zoom', 'teams']}}]


In [8]:
# invalid platform
try:
    meeting_info = client.request_bot(
        platform="zoom",
        native_meeting_id=native_meeting_id,
        bot_name="Vomeet's bot",
        language='es',
        task = 'transcribe'
    )
except Exception as e:
    print(e)
#'msg': "HTTP Error 422 for POST http://localhost:18056/bots: Invalid platform/native_meeting_id combination: cannot construct meeting URL"

HTTP Error 422 for POST http://localhost:18056/bots: Invalid platform/native_meeting_id combination: cannot construct meeting URL


In [9]:
# invalid language
try:
    meeting_info = client.request_bot(
        platform=platform,
        native_meeting_id=native_meeting_id,
        bot_name="Vomeet's bot",
        language='invalid language',
        task = 'transcribe'
    )
except Exception as e:
    print(e)
#'msg':'loc': ['body', 'language'], 'msg': "Invalid language code 'invalid language

HTTP Error 422 for POST http://localhost:18056/bots: [{'loc': ['body', 'language'], 'msg': "Invalid language code 'invalid language'. Must be one of: ['af', 'am', 'ar', 'as', 'az', 'ba', 'be', 'bg', 'bn', 'bo', 'br', 'bs', 'ca', 'cs', 'cy', 'da', 'de', 'el', 'en', 'es', 'et', 'eu', 'fa', 'fi', 'fo', 'fr', 'gl', 'gu', 'ha', 'haw', 'he', 'hi', 'hr', 'ht', 'hu', 'hy', 'id', 'is', 'it', 'ja', 'jw', 'ka', 'kk', 'km', 'kn', 'ko', 'la', 'lb', 'ln', 'lo', 'lt', 'lv', 'mg', 'mi', 'mk', 'ml', 'mn', 'mr', 'ms', 'mt', 'my', 'ne', 'nl', 'nn', 'no', 'oc', 'pa', 'pl', 'ps', 'pt', 'ro', 'ru', 'sa', 'sd', 'si', 'sk', 'sl', 'sn', 'so', 'sq', 'sr', 'su', 'sv', 'sw', 'ta', 'te', 'tg', 'th', 'tk', 'tl', 'tr', 'tt', 'uk', 'ur', 'uz', 'vi', 'yi', 'yo', 'yue', 'zh']", 'type': 'value_error'}]


In [10]:
# invalid task
try:
    meeting_info = client.request_bot(
        platform=platform,
        native_meeting_id=native_meeting_id,
        bot_name="Vomeet's bot",
        language='en',
        task = 'invalid task'
    )
except Exception as e:
    print(e)
#'msg':[{'loc': ['body', 'language'], 'msg': "Invalid language code 'invalid language'.

HTTP Error 422 for POST http://localhost:18056/bots: [{'loc': ['body', 'task'], 'msg': "Invalid task 'invalid task'. Must be one of: ['transcribe', 'translate']", 'type': 'value_error'}]


In [ ]:
# OK
meeting_info = client.request_bot(
    platform=platform,
    native_meeting_id=native_meeting_id,
    bot_name="Vomeet's bot",
    language='en',
    task = 'transcribe'
)


In [12]:
while True:
    transcript = client.get_transcript(native_meeting_id=native_meeting_id,platform='google_meet')
    df = pd.DataFrame(transcript['segments'])#.sort_values(['created_at','start'])#.drop_duplicates('start',keep='last')
    clear_output()
    display(df.sort_values('absolute_start_time').tail(10))
    time.sleep(1)

,start,end,text,language,created_at,speaker,absolute_start_time,absolute_end_time
0,0.0,1.393,"Então, isso é sinhoso, isso é sinhoso, isso é...",pt,None,Dmitriy Grankin,2025-10-03T11:07:39.722458+00:00,2025-10-03T11:07:41.115458+00:00


KeyboardInterrupt: 

In [13]:
# invalid platform
try:
    client.stop_bot(platform="invalid platform",native_meeting_id=native_meeting_id)
except Exception as e:
    print(e)
#'msg':  'msg': "value is not a valid enumeration member; permitted

HTTP Error 422 for DELETE http://localhost:18056/bots/invalid platform/fjp-cbnn-hug: [{'loc': ['path', 'platform'], 'msg': "value is not a valid enumeration member; permitted: 'google_meet', 'zoom', 'teams'", 'type': 'type_error.enum', 'ctx': {'enum_values': ['google_meet', 'zoom', 'teams']}}]


In [ ]:
# invalid meeting id
try:
    client.stop_bot(platform=platform,native_meeting_id='invalid meeting id')
except Exception as e:
    print(e)
#'msg':  'msg': "value is not a valid enumeration member; permitted

HTTP Error 404 for DELETE http://localhost:18056/bots/google_meet/invalid meeting id: No meeting found to stop.


In [15]:
running_bots = client.get_running_bots_status()

In [16]:
# invalid meeting id
try:
    client.update_bot_config(
        platform="invalid platform",
        native_meeting_id=native_meeting_id,
        language='en',
        task = 'transcribe'
    )
except Exception as e:
    print(e)
#'msg':  'msg': "value is not a valid enumeration member; permitted

HTTP Error 422 for PUT http://localhost:18056/bots/invalid platform/fjp-cbnn-hug/config: [{'loc': ['path', 'platform'], 'msg': "value is not a valid enumeration member; permitted: 'google_meet', 'zoom', 'teams'", 'type': 'type_error.enum', 'ctx': {'enum_values': ['google_meet', 'zoom', 'teams']}}]


In [ ]:
# native_meeting_id
try:
    client.update_bot_config(
        platform=platform,
        native_meeting_id='invalid meeting id',
        language='en',
        task = 'transcribe'
    )
except Exception as e:
    print(e)
#'msg':  'msg': "o active meeting found 

HTTP Error 404 for PUT http://localhost:18056/bots/google_meet/invalid meeting id/config: No active meeting found for platform google_meet and meeting ID invalid meeting id.


In [ ]:
# invalid task
try:
    client.update_bot_config(
        platform=platform,
        native_meeting_id=native_meeting_id,
        language='en',
        task = 'invalid task'
    )
except Exception as e:
    print(e)
#'msg':  'msg': 'msg': "Invalid task 'invalid task'. Must be one of: ['transcribe', 'translate']",

HTTP Error 422 for PUT http://localhost:18056/bots/google_meet/fjp-cbnn-hug/config: [{'loc': ['body', 'task'], 'msg': "Invalid task 'invalid task'. Must be one of: ['transcribe', 'translate']", 'type': 'value_error'}]


In [ ]:
# OK

client.update_bot_config(
    platform=platform,
    native_meeting_id=native_meeting_id,
    language='en'
)

In [21]:
while True:
    transcript = client.get_transcript(native_meeting_id=native_meeting_id,platform=platform)
    df = pd.DataFrame(transcript['segments'])#.sort_values(['created_at','start'])#.drop_duplicates('start',keep='last')
    clear_output()
    display(df.sort_values('absolute_start_time').tail(10))
    time.sleep(1)

,start,end,text,language,created_at,speaker,absolute_start_time,absolute_end_time
0,0.0,22.72,"Estaba adaptando phase 4, phase 6, 9, 9, 8, 9...",es,None,Teams Participant (participant-avatar),2025-10-03T10:28:05.100406+00:00,2025-10-03T10:28:27.820406+00:00


KeyboardInterrupt: 

In [43]:
running_bots = client.get_running_bots_status()

In [44]:
running_bots

[{'container_id': 'b8297a8146595238403ffe5623fb1d348749f571ea5ce712afbec92a73d370a3',
  'container_name': 'vomeet-bot-98-c947eddd',
  'platform': 'teams',
  'native_meeting_id': 'https://teams.live.com/meet/9399697580372?p=A5AU2r3RVJDzsCajcA',
  'status': 'Up 3 minutes',
  'normalized_status': 'Up',
  'created_at': '2025-10-03T10:29:17+00:00',
  'labels': {'desktop.docker.io/ports.scheme': 'v2',
   'org.opencontainers.image.ref.name': 'ubuntu',
   'org.opencontainers.image.version': '22.04',
   'vomeet.user_id': '115'},
  'meeting_id_from_name': '98'}]

In [19]:
client.stop_bot(platform=platform,native_meeting_id=native_meeting_id)


{'message': 'Stop request accepted and is being processed.'}

In [37]:
meetings = client.get_meetings()

In [ ]:
client.update_meeting_data(
    platform=platform,
    native_meeting_id=native_meeting_id,
    name="Test Meeting",
)

In [ ]:
client.delete_meeting(
    platform=platform,
    native_meeting_id=native_meeting_id,
)